<a href="https://colab.research.google.com/github/brownnyi/Project_FakeNews-Detector/blob/main/BERT_model_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
!pip install transformers

In [ ]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

In [ ]:
n_devices = torch.cuda.device_count()
print(n_devices)

for i in range(n_devices):
    print(torch.cuda.get_device_name(i))

1
Tesla T4


In [ ]:
df = pd.read_csv('drive/MyDrive/멀티캠퍼스_5회차/data/p1_EC.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42063 entries, 0 to 42062
Data columns (total 17 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   Unnamed: 0                         42063 non-null  int64 
 1   sourceDataInfo_newsID              42063 non-null  object
 2   sourceDataInfo_newsCategory        42063 non-null  object
 3   sourceDataInfo_newsSubcategory     41853 non-null  object
 4   sourceDataInfo_newsTitle           42063 non-null  object
 5   sourceDataInfo_newsSubTitle        24578 non-null  object
 6   sourceDataInfo_newsContent         42063 non-null  object
 7   sourceDataInfo_partNum             42063 non-null  object
 8   sourceDataInfo_useType             42063 non-null  int64 
 9   sourceDataInfo_processType         42063 non-null  object
 10  sourceDataInfo_processPattern      42063 non-null  int64 
 11  sourceDataInfo_processLevel        42063 non-null  object
 12  sour

In [ ]:
df.sourceDataInfo_useType.value_counts()

0    21399
1    20664
Name: sourceDataInfo_useType, dtype: int64

In [ ]:
df.labeledDataInfo_click

In [ ]:
df_test = df.iloc[:,[6,14,15]]

In [ ]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42063 entries, 0 to 42062
Data columns (total 3 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   sourceDataInfo_newsContent      42063 non-null  object
 1   labeledDataInfo_newTitle        42063 non-null  object
 2   labeledDataInfo_clickbaitClass  42063 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 986.0+ KB


In [ ]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 64.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 47.3 MB/s eta 0:00:00


In [ ]:
#토큰화한 제목과 본문의 유사도에 따른 거짓판별 모델

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

from konlpy.tag import Okt
okt = Okt()

df_test['tokened_title'] = df_test['labeledDataInfo_newTitle'].apply(lambda x: okt.morphs(x))
df_test['tokened_content'] = df_test['sourceDataInfo_newsContent'].apply(lambda x: okt.morphs(x))

<ipython-input-13-94866fc09504>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['tokened_title'] = df_test['labeledDataInfo_newTitle'].apply(lambda x: okt.morphs(x))
<ipython-input-13-94866fc09504>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['tokened_content'] = df_test['sourceDataInfo_newsContent'].apply(lambda x: okt.morphs(x))


In [ ]:
df_test

,sourceDataInfo_newsContent,labeledDataInfo_newTitle,labeledDataInfo_clickbaitClass,tokened_title,tokened_content
0,"\""사죄한다.\"" 운전기사들에게 허구한 날 폭언을 퍼부어 윤리 문제가 불거진 이장한...",'친환경 금융 목표'…'탄소중립' 실천 나선 금융업계,0,"[', 친환경, 금융, 목표, '…', 탄소, 중립, ', 실천, 나선, 금융업, 계]","[\"", 사죄, 한다, .\"", 운전기사, 들, 에게, 허구, 한, 날, 폭언, 을..."
1,"최종구 금융위원회 위원장이 \""'임팩트금융(사회적금융)'의 안착 차원에서 정부가 직...",공동주택 하자분쟁 조정 본격화,0,"[공동, 주택, 하자, 분쟁, 조정, 본격, 화]","[최종, 구, 금융, 위원회, 위원장, 이, \""', 임팩트, 금융, (, 사회, ..."
2,"한국기업지배구조원이 8월 31일 국내 상장회사의 환경경영(E), 사회책임경영(S),...","LG생활건강, 사상 최대 실적에도 주가 반등은 '글쎄'",0,"[LG, 생활, 건강, ,, 사상, 최대, 실적, 에도, 주가, 반등, 은, ', ...","[한국, 기업, 지배구조, 원, 이, 8월, 31일, 국내, 상장회사, 의, 환경,..."
3,요즘 프로보노 형 사회 공헌 활동을 펴는 회사가 늘고 있다.\n회사를 경영하거나 기...,"로봇청소기 강자 코오롱글로벌, '드리미 W10' 국내 론칭",0,"[로봇청소기, 강, 자, 코오롱, 글로벌, ,, ', 드리, 미, W, 10, ',...","[요즘, 프로보노, 형, 사회, 공헌, 활동, 을, 펴는, 회사, 가, 늘, 고, ..."
4,"런던 현지 시각으로 7일, 스탠다드앤푸어스 다우존스 인디시즈와 로베코샘이 다우존스지...","한숨만 쉬는 유통업계, 매출 얼마나 줄었나 봤더니…",0,"[한숨, 만, 쉬는, 유통업, 계, ,, 매출, 얼마나, 줄었나, 봤더니, …]","[런던, 현지, 시각, 으로, 7일, ,, 스탠다드, 앤, 푸, 어스, 다우존스, ..."
...,...,...,...,...,...
42058,올해 2분기 수출이 전년 동기 대비 10% 안팎 증가할 것이란 전망이 나왔다.\n한...,"수은 \""2분기 수출 전년比 10% 내외 증가\""",1,"[수은, \"", 2분, 기, 수출, 전, 년, 比, 10%, 내외, 증가, \""]","[올해, 2분, 기, 수출, 이, 전, 년, 동기, 대비, 10%, 안팎, 증가, ..."
42059,날씨 앱 첫화면날씨의 개발사 티니어(TNEAR)가 우크라이나의 평화를 기원하며 뜻 ...,"(주)티니어, 우크라이나 전쟁 고아를 위한 기부 동참",1,"[(, 주, ), 티니, 어, ,, 우크라이나, 전쟁, 고아, 를, 위, 한, 기부...","[날씨, 앱, 첫, 화면, 날씨, 의, 개발, 사, 티니, 어, (, TNEAR, ..."
42060,코로나19 발생으로 타격을 받은 자영업자를 대상으로 한국형 실업부조인 국민취업지원제...,"KDI \""코로나로 자영업자 고용안전망 한계…실업부조 강화해야\""",1,"[KDI, \"", 코로나, 로, 자영, 업자, 고용, 안전, 망, 한계, …, 실업...","[코로나, 19, 발생, 으로, 타격, 을, 받은, 자영, 업자, 를, 대상, 으로..."
42061,올해 1분기 자산유동화증권(ABS) 발행 총액은 전년 동기 대비 20.2% 감소한 ...,"1Q ABS 발행 전년 대비 20.2% ↓…\""주택금융공사 MBS 발행 축소 영향\""",1,"[1, Q, ABS, 발행, 전, 년, 대비, 20.2%, ↓, …\"", 주택, 금...","[올해, 1분, 기, 자산, 유동화, 증권, (, ABS, ), 발행, 총, 액, ..."


In [ ]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42063 entries, 0 to 42062
Data columns (total 5 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   sourceDataInfo_newsContent      42063 non-null  object
 1   labeledDataInfo_newTitle        42063 non-null  object
 2   labeledDataInfo_clickbaitClass  42063 non-null  int64 
 3   tokened_title                   42063 non-null  object
 4   tokened_content                 42063 non-null  object
dtypes: int64(1), object(4)
memory usage: 1.6+ MB


In [ ]:
df_test['test_text'] = df_test['tokened_title'].apply(lambda x: ' '.join(map(str, x))) + ' ' + df_test['tokened_content'].apply(lambda x: ' '.join(map(str, x)))

<ipython-input-17-fccb8e821f43>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['test_text'] = df_test['tokened_title'].apply(lambda x: ' '.join(map(str, x))) + ' ' + df_test['tokened_content'].apply(lambda x: ' '.join(map(str, x)))


In [ ]:
df_test

,sourceDataInfo_newsContent,labeledDataInfo_newTitle,labeledDataInfo_clickbaitClass,tokened_title,tokened_content,test_text
0,"\""사죄한다.\"" 운전기사들에게 허구한 날 폭언을 퍼부어 윤리 문제가 불거진 이장한...",'친환경 금융 목표'…'탄소중립' 실천 나선 금융업계,0,"[', 친환경, 금융, 목표, '…', 탄소, 중립, ', 실천, 나선, 금융업, 계]","[\"", 사죄, 한다, .\"", 운전기사, 들, 에게, 허구, 한, 날, 폭언, 을...","' 친환경 금융 목표 '…' 탄소 중립 ' 실천 나선 금융업 계 \"" 사죄 한다 ...."
1,"최종구 금융위원회 위원장이 \""'임팩트금융(사회적금융)'의 안착 차원에서 정부가 직...",공동주택 하자분쟁 조정 본격화,0,"[공동, 주택, 하자, 분쟁, 조정, 본격, 화]","[최종, 구, 금융, 위원회, 위원장, 이, \""', 임팩트, 금융, (, 사회, ...","공동 주택 하자 분쟁 조정 본격 화 최종 구 금융 위원회 위원장 이 \""' 임팩트 ..."
2,"한국기업지배구조원이 8월 31일 국내 상장회사의 환경경영(E), 사회책임경영(S),...","LG생활건강, 사상 최대 실적에도 주가 반등은 '글쎄'",0,"[LG, 생활, 건강, ,, 사상, 최대, 실적, 에도, 주가, 반등, 은, ', ...","[한국, 기업, 지배구조, 원, 이, 8월, 31일, 국내, 상장회사, 의, 환경,...","LG 생활 건강 , 사상 최대 실적 에도 주가 반등 은 ' 글쎄 ' 한국 기업 지배..."
3,요즘 프로보노 형 사회 공헌 활동을 펴는 회사가 늘고 있다.\n회사를 경영하거나 기...,"로봇청소기 강자 코오롱글로벌, '드리미 W10' 국내 론칭",0,"[로봇청소기, 강, 자, 코오롱, 글로벌, ,, ', 드리, 미, W, 10, ',...","[요즘, 프로보노, 형, 사회, 공헌, 활동, 을, 펴는, 회사, 가, 늘, 고, ...","로봇청소기 강 자 코오롱 글로벌 , ' 드리 미 W 10 ' 국내 론칭 요즘 프로보..."
4,"런던 현지 시각으로 7일, 스탠다드앤푸어스 다우존스 인디시즈와 로베코샘이 다우존스지...","한숨만 쉬는 유통업계, 매출 얼마나 줄었나 봤더니…",0,"[한숨, 만, 쉬는, 유통업, 계, ,, 매출, 얼마나, 줄었나, 봤더니, …]","[런던, 현지, 시각, 으로, 7일, ,, 스탠다드, 앤, 푸, 어스, 다우존스, ...","한숨 만 쉬는 유통업 계 , 매출 얼마나 줄었나 봤더니 … 런던 현지 시각 으로 7..."
...,...,...,...,...,...,...
42058,올해 2분기 수출이 전년 동기 대비 10% 안팎 증가할 것이란 전망이 나왔다.\n한...,"수은 \""2분기 수출 전년比 10% 내외 증가\""",1,"[수은, \"", 2분, 기, 수출, 전, 년, 比, 10%, 내외, 증가, \""]","[올해, 2분, 기, 수출, 이, 전, 년, 동기, 대비, 10%, 안팎, 증가, ...","수은 \"" 2분 기 수출 전 년 比 10% 내외 증가 \"" 올해 2분 기 수출 이 ..."
42059,날씨 앱 첫화면날씨의 개발사 티니어(TNEAR)가 우크라이나의 평화를 기원하며 뜻 ...,"(주)티니어, 우크라이나 전쟁 고아를 위한 기부 동참",1,"[(, 주, ), 티니, 어, ,, 우크라이나, 전쟁, 고아, 를, 위, 한, 기부...","[날씨, 앱, 첫, 화면, 날씨, 의, 개발, 사, 티니, 어, (, TNEAR, ...","( 주 ) 티니 어 , 우크라이나 전쟁 고아 를 위 한 기부 동참 날씨 앱 첫 화면..."
42060,코로나19 발생으로 타격을 받은 자영업자를 대상으로 한국형 실업부조인 국민취업지원제...,"KDI \""코로나로 자영업자 고용안전망 한계…실업부조 강화해야\""",1,"[KDI, \"", 코로나, 로, 자영, 업자, 고용, 안전, 망, 한계, …, 실업...","[코로나, 19, 발생, 으로, 타격, 을, 받은, 자영, 업자, 를, 대상, 으로...","KDI \"" 코로나 로 자영 업자 고용 안전 망 한계 … 실업 부조 강화해야 \"" ..."
42061,올해 1분기 자산유동화증권(ABS) 발행 총액은 전년 동기 대비 20.2% 감소한 ...,"1Q ABS 발행 전년 대비 20.2% ↓…\""주택금융공사 MBS 발행 축소 영향\""",1,"[1, Q, ABS, 발행, 전, 년, 대비, 20.2%, ↓, …\"", 주택, 금...","[올해, 1분, 기, 자산, 유동화, 증권, (, ABS, ), 발행, 총, 액, ...","1 Q ABS 발행 전 년 대비 20.2% ↓ …\"" 주택 금융 공사 MBS 발행 ..."


In [ ]:
import numpy as np

# CountVectorizer를 사용하여 토큰을 벡터로 변환
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df_test['test_text'])

# 벡터 간 코사인 유사도 계산
cosine_sim = cosine_similarity(X)
title_len = np.mean([len(i) for i in df_test['tokened_title']])
normalized_similarity = cosine_sim / title_len

# 낚시성 여부 판별 (임계값은 실험적으로 조정)
threshold = 0.8
df_test['fishing_prediction'] = cosine_sim[:, 0] > threshold

In [ ]:
# 훈련 및 평가 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, data['labeledDataInfo_clickbaitClass'], test_size=0.2, random_state=42)

# RandomForestClassifier를 사용하여 모델 훈련
model = RandomForestClassifier()
model.fit(X_train, y_train)

# 테스트 데이터로 모델 평가
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f'모델 정확도: {accuracy}')
